In [ ]:
import plotly.io as pio
import logging
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
import numpy as np
from plotly.colors import qualitative as pc
pio.renderers.default = 'notebook'

class Stockanalyzer:
    def __init__(self,file=None,freq='yearly',interval='1d'):
        self.file = file
        self.logger = logging.getLogger(__name__)
        self.freq = freq if freq in ['quarterly','yearly'] else None
        self.stocks = self._load_tickers() if file else []
        self.interval = interval if interval in ['1d','1wk','1mo'] else None
        
    def _load_tickers(self):
        try:
            with open(self.file,'r') as f:
                return [i.strip() for i in f.readlines()]
        except FileNotFoundError:
            self.logger.error(f'{self.file} not found in cache')
            return []
    
    def get_data(self):
        if not self.stocks or self.freq is None or self.interval is None:
            return False
        for ticker in self.stocks:
            pathf = f'{ticker}_financials_{self.freq}.csv'
            patht = f'{ticker}_{self.interval}.csv'
            try:
                dff = pd.read_csv(pathf,index_col=0)
                dft = pd.read_csv(patht,index_col=0)
                self.logger.info(f'{ticker} found in cache')
            except FileNotFoundError:
                try:
                    stock = yf.Ticker(ticker)
                    cf = stock.get_cashflow(freq=self.freq)
                    inc = stock.get_incomestmt(freq=self.freq)
                    dff = pd.concat([cf,inc],join='inner',axis=0)
                    if self.freq == 'quarterly':
                        dff.columns = dff.columns.to_period('Q').astype('str')
                    else:
                        dff.columns = dff.columns.year.astype('str')
                    dft = yf.download(ticker,interval=self.interval,period='max',auto_adjust=True,progress=False)
                    dft.columns = ['Open','High','Low','Close','Volume']
                    dff.to_csv(pathf)
                    dft.to_csv(patht)
                except ValueError:
                    self.logger.warning(f'{ticker} not available')
                    continue
        return True
        
    def plot_profitability(self):
        if not self.stocks or self.freq is None:
            return False
        for ticker in self.stocks:
            try:
                df = pd.read_csv(f'{ticker}_financials_{self.freq}.csv',index_col=0)
                df = df.loc[['TotalRevenue','NetIncome','FreeCashFlow'],::-1].copy().dropna(axis=1)
            except (FileNotFoundError,KeyError):
                self.logger.warning(f'{ticker} not found in cache')
                continue
            colors = pc.Plotly
            fig = go.Figure()
            for j, i in enumerate(df.index.drop('TotalRevenue')):
                fig.add_trace(
                    go.Bar(
                        x=df.columns,
                        y=df.loc[i]/df.loc['TotalRevenue']*100,
                        name=i+'Margin',
                        marker=dict(color=colors[j],line=dict(color='#fff',width=1)),
                        hovertext=i+'Margin',
                        hovertemplate='<b>%{x}</b><br>%{y:,.0f}%',
                        width=0.38
                    )
                )
            fig.add_trace(
                go.Scatter(
                    x=df.columns,
                    y=(df.loc['TotalRevenue'].pct_change().fillna(0.0)*100),
                    name='RevenueSlope',
                    mode='lines+markers',
                    line=dict(color=colors[2],width=3,shape='spline'),
                    marker=dict(size=11,symbol='diamond',color=colors[2],line=dict(color='#fff',width=2)),
                    hovertext='ChangeInRevenue',hovertemplate='<b>%{x}</b><br>%{y:.2}%'
                )
            )
            fig.update_layout(
                plot_bgcolor='#000',
                paper_bgcolor='#000',
                xaxis=dict(gridcolor='#333'),
                yaxis=dict(gridcolor='#333',title='%'),
                font=dict(color='#fff',size=12),
                title=dict(text=f'{ticker} Profitability & Momentum',font=dict(weight='bold',size=16))
            )
            fig.show()
        return True
    
    def print_pe(self):
        if not self.stocks or self.freq is None or self.interval is None:
            return False
        peratio = pd.DataFrame()
        for ticker in self.stocks:
            try:
                try:
                    stock = yf.Ticker(ticker)
                    fpe = stock.info['forwardPE']
                except (ValueError,KeyError):
                    fpe = np.NaN
                    self.logger.warning(f'No Forward PE for {ticker} available')
                    continue
                if self.freq == 'quarterly':
                    deps_ttm = pd.read_csv(f'{ticker}_financials_{self.freq}.csv',index_col=0).loc['DilutedEPS'].iloc[-4:].sum()
                else:
                    deps_ttm = pd.read_csv(f'{ticker}_financials_{self.freq}.csv',index_col=0).loc['DilutedEPS'].iloc[-1]
                price = pd.read_csv(f'{ticker}_{self.interval}.csv',index_col=0,parse_dates=True)['Close'].iloc[-1]
                pe = round(price/deps_ttm,2)
                peratio.loc[ticker,'PE_ttm'] = pe if pe >= 0 else np.NaN
                peratio.loc[ticker,'PE_forward'] = round(fpe,2) if fpe >= 0 else np.NaN
                peratio.loc[ticker,'Discount_pct'] = round((1-fpe/pe)*100,2) if pe >= 0 else np.NaN
            except FileNotFoundError:
                self.logger.warning(f'No sufficient data for {ticker} to calculate P/E')
                continue
        print(peratio.sort_values(by='Discount_pct',ascending=False))
    
    def __call__(self):
        if self.get_data():
            self.print_pe()
            self.plot_profitability()
    
if __name__ == '__main__':
    logging.basicConfig(level=logging.WARNING)
    pf = Stockanalyzer('mag7.txt','quarterly')
    pf()